# Filtracion de datos

In [2]:
import sqlite3
import pandas as pd
import re

In [3]:
# Esta función nos sirve para ir monitoreando los filtros
def resumen_filtrado(df, len_original, texto=''):
    len_nuevo = len(df)
    eliminadas = len_original - len_nuevo
    porcentaje = (len_nuevo / len_original) * 100 if len_original > 0 else 0
    print("Filtrado: {:6d} → {:<6d} filas ({:6.2f}% mantenidas, {} eliminadas){}".format(
        len_original, len_nuevo, porcentaje, eliminadas,
        f'   |   {texto}' if texto else ''
    ))

# Esta sirve para monitorear los cambios
def resumen_cambios(df2, df1):
    sentinel = object()
    a = df1.astype(object).where(~df1.isna(), sentinel)
    b = df2.astype(object).where(~df2.isna(), sentinel)

    iguales_mask = (a == b).all(axis=1)
    iguales = int(iguales_mask.sum())
    total = len(df1)
    diferentes = total - iguales
    p = (iguales / total * 100) if total else 0.0

    print(f"Comparación de DataFrames ({total:6d} filas totales)")
    print(f"Filas modificadas : {diferentes:6d}  ({100-p:6.2f}% distintas)")


Nuestras bases de datos, tanto la de arriendos como la de ventas contienen 4 tablas.
'principal', 'full_specs', 'full_specs_bin' son tablas con una columna que funciona como identificador de la propiedad llamada mlc, y el resto de las columnas son características de la propiedad. La cuarta tabla se llama 'grupos_bin' y simplemente dice a que tipo de grupo corresponde cada una de las características binarias de las propiedades. 

En la siguiente celda convertimos las 3 tablas que contienen información de las propiedades en DataFrames.

In [5]:
db_path = "database_venta.db" # "database_venta.db"
tablas_nombres = ('principal', 'full_specs', 'full_specs_bin', 'grupos_bin')

conn  = sqlite3.connect(db_path)
df_p, df_fs, df_fsb, df_gb = [pd.read_sql_query(f"SELECT * FROM {tab}", conn) for tab in tablas_nombres]
conn.close()


last_len = len(df_p)
print(f'principal → {len(df_p)}, full_specs → {len(df_fs)}, full_specs_bin → {len(df_fsb)}')

DatabaseError: Execution failed on sql 'SELECT * FROM principal': no such table: principal

# Filtrar DataFrame principal
Acá vamos a hacer filtros que reducirán el dataframe principal. Hay que notar que este es el filtrado mas importante porque este datatrame no solo tiene los datos mas relevantes y mas fiables, sino que tambien es la única tabla donde si o si se encuentran el 100% de las propiedades. las otras tablas no necesariamente contienen todas las propiedades, pues puede haber algune aque no tenga definidas características binarias (por ende no tendrá ninguna fila en la tabla 'full_specs_bin') o puede ser que no se hayan scrapeado correctamente esos elementos.




****Filtro 1:**** Eliminar propiedades sin superficie total o sin baños o sin direcición

In [122]:
df_p1 = df_p.dropna(subset=["Superficie total", "Superficie total unidad", "Baños", "Dirección"])
resumen_filtrado(df_p1, len(df_p))

Filtrado: 104545 → 100483 filas ( 96.11% mantenidas, 4062 eliminadas)


****Filtro  2:**** Eliminar propiedades con mas de el dobre de baños que de dormitorios (sin dormitorios se reemplaza por 1)

In [123]:
df_p2 = df_p1[df_p1["Baños"] < 2 * df_p1["Dormitorios"].fillna(1)]
resumen_filtrado(df_p2, len(df_p1))

Filtrado: 100483 → 99881  filas ( 99.40% mantenidas, 602 eliminadas)


****Filtro 3:**** Esto realmente no es un filtro. Lo que hacemos es reemplazar dormitorios nulos por 0 dormitorios. No se hizo antes para usar el .fillna(1) en el filtro 2.

In [124]:
df_p3 = df_p2.copy()
df_p3["Dormitorios"] = df_p3["Dormitorios"].fillna(0)
resumen_cambios(df_p3, df_p2)

Comparación de DataFrames ( 99881 filas totales)
Filas modificadas :    246  (  0.25% distintas)


****Filtro 4:**** Eliminamos cuando no hay dormitorio y no es departamento. Así filtramos ventas de terreno o arriendos de locales comerciales mal indexados.

In [126]:
df_p4 = df_p3[~((df_p3["Dormitorios"] == 0) & (df_p3['inmueble'] != 'departamento'))]
resumen_filtrado(df_p4, len(df_p3))


Filtrado:  99881 → 99846  filas ( 99.96% mantenidas, 35 eliminadas)


****Filtro 5:**** Elimina propiedades cuya superficie está medida en hectareas y es de mas de 50ha (eso ya no es una parcela, es una venta de terrenos mal indexada)

In [127]:
df_p5 = df_p4[~( (df_p4["Superficie total unidad"] == 'ha') & (df_p4["Superficie total"] > 50))]
resumen_filtrado(df_p5, len(df_p4))


Filtrado:  99846 → 99796  filas ( 99.95% mantenidas, 50 eliminadas)


****Filtro 6:**** Eliminamos propiedades de mas de 10 habitaciones o de mas de 10 baños, pues seguramente son edificios compretos o casas con finalidad comercial u hotelera mal indexadas

In [128]:
df_p6 = df_p5[(df_p5["Dormitorios"].fillna(0) < 10) & (df_p5["Baños"].fillna(0) < 10)]
resumen_filtrado(df_p6, len(df_p5))


Filtrado:  99796 → 99293  filas ( 99.50% mantenidas, 503 eliminadas)


****Filtro 7:**** Este filtro elimina los elementos que no contienen precio ni en pesos ni en uf

In [130]:
df_p7 = df_p6[df_p6["$"].notna() | df_p6["UF"].notna()]
resumen_filtrado(df_p7, len(df_p6))

Filtrado:  99293 → 99293  filas (100.00% mantenidas, 0 eliminadas)


****Filtro 8:**** Eliminamos las que tienen precio en US$ (suelen ser propiedades de fuera de chile mal catalogadas)

In [131]:
df_p8 = df_p7[df_p7["US$"].isna()]
resumen_filtrado(df_p8, len(df_p7))

Filtrado:  99293 → 99284  filas ( 99.99% mantenidas, 9 eliminadas)


****Filtro 9:**** Acá completamos los precios, pues algunas propiedades tienen el precio en Uf's pero no en pesos chilenos

In [132]:
uf_cpl = 39541.62

df_p9 = df_p8.copy()
df_p9.loc[df_p9["$"].isna() & df_p9["UF"].notna(), "$"] = (df_p9.loc[df_p9["$"].isna() & df_p9["UF"].notna(), "UF"] * uf_cpl)
resumen_cambios(df_p9, df_p8)

Comparación de DataFrames ( 99284 filas totales)
Filas modificadas :      0  (  0.00% distintas)


****Filtro 10:**** Eliminamos cuando los gastos comunes son mayores al valor de la propiedad (ya sea arriendo o venta)

In [133]:
df_p10 = df_p9[df_p9["gastos_comunes"].fillna(0) <= df_p9["$"].fillna(0)]
resumen_filtrado(df_p10, len(df_p9))

Filtrado:  99284 → 99270  filas ( 99.99% mantenidas, 14 eliminadas)


****Filtro 11:**** Eliminamos cuando los valores son muy baratos o muy caros

In [136]:
M = 1e6
if 'venta' in db_path:
    df_p11 = df_p10[(21*M <= df_p10['$']) & df_p10['$'] <= 6000*M]
elif 'arriendo' in db_path:
    df_p11 = df_p10[
        (df_p10['$'] >= 100000) &
        ~( (df_p10['$'] > 10*M) & (df_p10['inmueble'] == 'departamento') ) &
        (df_p10['$'] <= 30*M)
    ]
resumen_filtrado(df_p11, len(df_p10))

Filtrado:  99270 → 99270  filas (100.00% mantenidas, 0 eliminadas)


****Filtro 12:**** Evitar venta de terrenos y de promesas de compra

In [137]:
df_p12 = df_p11.copy()
for frase in {
    'Se vende terreno', 'Se vende terreno'.lower(), 'Se vende terreno'.upper(),
    'promesa de compra', 'promesa compra', 'Promesa Compra'
}:
    df_p12 = df_p12[~df_p12['titulo'].str.contains(frase, case=False, na=False)]
    df_p12 = df_p12[~df_p12['Descripción'].str.contains(frase, case=False, na=False)]
resumen_filtrado(df_p12, len(df_p11))

Filtrado:  99270 → 99020  filas ( 99.75% mantenidas, 250 eliminadas)


****Filtro 13 (Exclusivo arriendos):**** Muchas veces, en especial en departamentos con arriendos bajo los $800.000, sucede que se promociona un precio, pero en la descripción pone que despues del segundo mes, este aumenta considerablemente (al rededor de un 100%). Para solucionar esto, se buscan distintos precios en la descripción, y en caso de que este estre entre 1.5 y 2.5 del precio publicado y sea menor al millón de pesos, se procede a evaluar si la descripción contiene la palabra garantía (para evitar confundir garantia con precio real). En caso de que pase todos estos filtros *una sola vez*, se procede a actualizar el valor real del arriendo. En caso de que pase los filtros *mas de 2 veces* se procede a eliminar esa fila del dataframe, pues no es confiable el precio y puede dañar las estadísticas.

Como se puede observar este filtro no es perfecto y lo iremos perfeccionando, pero lo revisamos exaustivamente printeando links y comparando y llegamos a la conclusión de que no genera falsos positivos, por lo que en un futuro lo iremos mejorando para que cada vez mas precios se reflejen correctamente en la estadística.

In [138]:
def extraer_precios(texto):
        if not isinstance(texto, str): return []
        patron = r'\$?\s*(?:\d{1,3}(?:\.\d{3})+|\d{6,})\b'

        # Esto es una lista de strings
        coincidencias = re.findall(patron, texto) 

        # Esto lo convierte en una lista de int's
        precios = [int(re.sub(r'[^\d]', '', c)) for c in coincidencias]

        # esto elimina numeros de telefono
        precios = [p for p in precios if len(str(p)) < 8]
        return precios




df_p13 = df_p12.copy()
del_idx = set()

if 'arriendo' in db_path:
    for i, fila in df_p13.iterrows():
        contador, numero_ = 0, None
        for numero in extraer_precios(fila['Descripción']):
            if 1.5 * fila['$'] < numero < 2.5 * fila['$'] and numero < 1e6:
                if not any([text in fila['Descripción'] for text in (
                    'Garantía', 'Garantia', 'garantía', 'garantia'
                )]):
                    contador += 1
                    numero_ = numero
            
        if contador == 1:
            df_p13.at[i, '$'] = numero_
        elif contador > 1:
            del_idx.add(i)
    
    

resumen_cambios(df_p13, df_p12)
print('----------------------------')
df_p13 = df_p13.drop(index=del_idx)
resumen_filtrado(df_p13, len(df_p12))

Comparación de DataFrames ( 99020 filas totales)
Filas modificadas :      0  (  0.00% distintas)
----------------------------
Filtrado:  99020 → 99020  filas (100.00% mantenidas, 0 eliminadas)


# Filtros de DataFrame full_specs

Este Dataframe contiene información mas completa sobre las propiedades, pero es ligeramente meos fiable. No necesariamente todas las propiedades tienen características full_specs, aunque la gran mayoría si.

Cabe destacar que si una propiedad Es filtrada en los filtros de principal, este filtro es tan inportante que amerita eliminar esa propiedad de la base de datos. Sin embargo, como los datos de full_specs tienen menos fiabilidad, muchas veces que los datos sean inconsistentes no implica que toda la propiedad lo sea, por lo que si se elimina la propiedad del dataframe full_specs, pero no de principal, y simpelmente se excluye esa propiedad en los analisis de variables que se encuentren en full_specs.

****Filtro 1:**** Iniciamos eliminando todas las filas cuyo mlc no se encuentra en principal, pues de ser así significa que esta propiedad fue filtrada antes.

In [139]:
df_fs1 = df_fs[df_fs['mlc'].isin(df_p13['mlc'])]
resumen_filtrado(df_fs1, len(df_fs))

Filtrado: 104178 → 99009  filas ( 95.04% mantenidas, 5169 eliminadas)


****Filtro 2:**** Acá eliminamos las filas donde haya inconsistencias en las medidas de la superficie.
1. total >= util
2. total >= terraza
3. total * gap >= util + terraza

In [140]:
# ! Filtro superficie
unidad, gap = {'m²': 1, 'ha': 10000, 0: 0}, 1
col1,  col2,  col3  = "Superficie total", "Superficie útil", "Superficie de terraza"
col1_, col2_, col3_ = [df_fs1[col + ' unidad'].map(unidad).fillna(0) for col in (col1, col2, col3)]

df_fs2 = df_fs1[
    (df_fs1[col1].fillna(0) * col1_             >= df_fs1[col2].fillna(0) * col2_) &
    (df_fs1[col1].fillna(0) * col1_             >= df_fs1[col3].fillna(0) * col3_) &
    (df_fs1[col1].fillna(0) * col1_ * (1 + gap) >= df_fs1[col2].fillna(0) * col2_ + df_fs1[col3].fillna(0) * col3_)
]
resumen_filtrado(df_fs2, len(df_fs1))

Filtrado:  99009 → 96604  filas ( 97.57% mantenidas, 2405 eliminadas)


****Filtro 3:**** Superficie util irreal (de las de 10.000 m2)

In [141]:
df_fs3 = df_fs2[df_fs2[col2].fillna(0) * col2_ <= 1e4] 
resumen_filtrado(df_fs3, len(df_fs2))

Filtrado:  96604 → 96498  filas ( 99.89% mantenidas, 106 eliminadas)


C:\Users\josep\AppData\Local\Temp\ipykernel_23276\4009217669.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_fs3 = df_fs2[df_fs2[col2].fillna(0) * col2_ <= 1e4]


****Filtro 4:**** Quitar gastos comunes en uf

In [142]:
df_fs4 = df_fs3[df_fs3['Gastos comunes unidad'].fillna(0) != 'UF'] 
resumen_filtrado(df_fs4, len(df_fs3))

Filtrado:  96498 → 95477  filas ( 98.94% mantenidas, 1021 eliminadas)


****Filtro 5:**** Quitar departamentos con mas de 5 estacionamientos.

****NOTA:**** Los otros filtros que hemos hecho en full_specs son filtros que implican desorden en los datos entregados en full_specs, pero no desorden en los datos generales de la propiedad. Sin embargo, que haya departamentos con mas de 5 estacionamientos si implica que la propiedad puede estar viciada perse, por lo que en este filtro eliminamos la propiedad tanto de principal como de full_specs.

In [ ]:
del_idx, del_mlc = set(), set()
for i, fila in df_fs4.iterrows():
    if df_p13.loc[df_p13["mlc"] == fila['mlc'], "inmueble"].item() == 'departamento' \
        and 6 <= fila['Estacionamientos']:
        del_idx.add(i), del_mlc.add(fila['mlc'])

df_fs5 = df_fs4.drop(index=del_idx)
df_p14 = df_p13[~df_p13['mlc'].isin(del_mlc)]

print('Filtro en full_specs')
resumen_filtrado(df_fs5, len(df_fs4))
print('---------------')
print('Filtro en principal')
resumen_filtrado(df_p14, len(df_p13))

Filtro en full_specs
Filtrado:  95477 → 95234  filas ( 99.75% mantenidas, 243 eliminadas)
---------------
Filtro en principal
Filtrado:  99020 → 98777  filas ( 99.75% mantenidas, 243 eliminadas)


# Filtros DataFrame full_specs_bin
Este dataframe contiene características binarias de las propiedades. No solo no todas las propiedades tienen características binarias, sino que estas son las características de menor calidad que tenemos. Por ende, al mínimo descuadre entre ellas, no hay que dudar en eliminar todas, pues puede corromper los análisis estadísticos.


****Filtro 1:**** Eliminamos todos los mlc que no se encuentran en full_specs, y por ende, tampoco en principal

In [144]:
df_fsb1 = df_fsb[df_fsb['mlc'].isin(df_fs5['mlc'])]
resumen_filtrado(df_fsb1, len(df_fsb))

Filtrado:  93151 → 87224  filas ( 93.64% mantenidas, 5927 eliminadas)


****Filtro 2:**** Este filtro elimina propiedades cuyas características binarias no sean consistentes lógicamente segun premisas que hemos generado.

In [ ]:
"""
Recepción       -> Conserjería    | Si no tiene recepción ¿Como va a tener conserjería?
Cisterna        -> Agua corriente | Sin cisterna no se puede tener agua corriente
Refrigerador    -> Cocina         | Ya es raro que un inmueble no tenga cocina,
                                  | pero si tiene refigeraror ya es de locos que no tenga cocina
Walk-in clóset  -> Closets        | Por definición un walk-in closen es un closet
Baño de visitas -> Living         | 
Uso comercial   -> ¬Solo familias | Si el uso en comercial no puede ser solo_familias
Solo familias   -> ¬Uso comercial | y viceversa
Jardín          -> (Patio v Con área verde) | un jardin no puede no estar en el patio (sino es un macetero)
                                              y  además, es un area verde por si mismo.
"""

df_fsb21 = df_fsb1[(df_fsb1["Recepción"]         != 1) | (df_fsb1["Conserjería"]     != 0)] if {"Recepción", "Conserjería"      }.issubset(df_fsb1.columns ) else df_fsb1
df_fsb22 = df_fsb21[(df_fsb21["Cisterna"]        != 1) | (df_fsb21["Agua corriente"] != 0)] if {"Cisterna", "Agua corriente"    }.issubset(df_fsb21.columns) else df_fsb21
df_fsb23 = df_fsb22[(df_fsb22["Refrigerador"]    != 1) | (df_fsb22["Cocina"]         != 0)] if {"Refrigerador", "Cocina"        }.issubset(df_fsb22.columns) else df_fsb22
df_fsb24 = df_fsb23[(df_fsb23["Walk-in clóset"]  != 1) | (df_fsb23["Closets"]        != 0)] if {"Walk-in clóset", "Closets"     }.issubset(df_fsb23.columns) else df_fsb23
df_fsb25 = df_fsb24[(df_fsb24["Baño de visitas"] != 1) | (df_fsb24["Living"]         != 0)] if {"Baño de visitas", "Living"     }.issubset(df_fsb24.columns) else df_fsb24
df_fsb26 = df_fsb25[(df_fsb25["Uso comercial"]   != 1) | (df_fsb25["Solo familias"]  != 1)] if {"Uso comercial", "Solo familias"}.issubset(df_fsb25.columns) else df_fsb25
df_fsb27 = df_fsb26[(df_fsb26["Solo familias"]   != 1) | (df_fsb26["Uso comercial"]  != 1)] if {"Uso comercial", "Solo familias"}.issubset(df_fsb26.columns) else df_fsb26
df_fsb2  = df_fsb27[(df_fsb27["Jardín"]          != 1) | (df_fsb27["Patio"]          != 0) | (df_fsb27["Con área verde"] != 0)] if {"Jardín", "Patio", "Con área verde"}.issubset(df_fsb27.columns) else df_fsb27


print('Resumen de cada uno de los filtros lógicos agregados')
print('---------------------------')
resumen_filtrado(df_fsb21, len(df_fsb1), 'Recepción -> Conserjería')
resumen_filtrado(df_fsb22, len(df_fsb21), 'Cisterna -> Agua corriente')
resumen_filtrado(df_fsb23, len(df_fsb22), 'Refrigerador -> Cocina ')
resumen_filtrado(df_fsb24, len(df_fsb23), 'Walk-in clóset -> Closets')
resumen_filtrado(df_fsb25, len(df_fsb24), 'Baño de visitas -> Living    ')
resumen_filtrado(df_fsb26, len(df_fsb25), 'Uso comercial -> ¬Solo familia')
resumen_filtrado(df_fsb27, len(df_fsb26), 'Solo familias -> ¬Uso comercial')
resumen_filtrado(df_fsb2, len(df_fsb27), 'Jardín -> (Patio v Con área verde)')
print('---------------------------')
print('Resumen acumulado de todos los filtros lógicos aplicados')
print('---------------------------')
resumen_filtrado(df_fsb2, len(df_fsb1))


Resumen de cada uno de los filtros lógicos agregados
---------------------------
Filtrado:  87224 → 84524  filas ( 96.90% mantenidas, 2700 eliminadas)   |   Recepción -> Conserjería
Filtrado:  84524 → 84288  filas ( 99.72% mantenidas, 236 eliminadas)   |   Cisterna -> Agua corriente
Filtrado:  84288 → 84124  filas ( 99.81% mantenidas, 164 eliminadas)   |   Refrigerador -> Cocina 
Filtrado:  84124 → 80860  filas ( 96.12% mantenidas, 3264 eliminadas)   |   Walk-in clóset -> Closets
Filtrado:  80860 → 75102  filas ( 92.88% mantenidas, 5758 eliminadas)   |   Baño de visitas -> Living    
Filtrado:  75102 → 75102  filas (100.00% mantenidas, 0 eliminadas)   |   Uso comercial -> ¬Solo familia
Filtrado:  75102 → 75102  filas (100.00% mantenidas, 0 eliminadas)   |   Solo familias -> ¬Uso comercial
Filtrado:  75102 → 70468  filas ( 93.83% mantenidas, 4634 eliminadas)   |   Jardín -> (Patio v Con área verde)
---------------------------
Resumen acumulado de todos los filtros lógicos aplicados
----

# Guardar base de datos filtrada

Ahora convetiremos estos dataframes en una base de datos .db.

In [1]:

db_path_clean = db_path[:-3] + '_clean.db'

dfs = {
    "principal": df_p,
    "full_specs": df_fs,
    "full_specs_bin": df_fsb,
    "grupos_bin": df_gb,
}


con = sqlite3.connect(db_path_clean)
for table, df in dfs.items():
    df.to_sql(table, con, if_exists="replace", index=False)
con.close()

NameError: name 'db_path' is not defined